<a href="https://colab.research.google.com/github/G-who0212/video_image_processing/blob/main/practice/cat_dog.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# 이미지 데이터를 코랩에서 다운받는 방법
# 1. 왼쪽의 파일트리에다가 업로드 하던가 -> 시간이 너무 오래 걸림..
# 2. 직접 다운받는 코드를 짜던가

import os
os.environ['KAGGLE_CONFIG_DIR'] = '/content/'

!kaggle competitions download -c dogs-vs-cats-redux-kernels-edition

 99% 803M/814M [00:05<00:00, 123MB/s]
100% 814M/814M [00:06<00:00, 142MB/s]


In [ ]:
!unzip -q /content/dogs-vs-cats-redux-kernels-edition.zip -d .

In [ ]:
!unzip -q train.zip -d .

In [ ]:
import os
print( len(os.listdir('/content/train/')))
#os.listdir('폴더') -> 폴더 안의 파일들을 리스트 안에 전부 담는다. [파일명1, 파일명2, 파일명3,...]

os.mkdir('/content/dataset')
os.mkdir('/content/dataset/cat')
os.mkdir('/content/dataset/dog')

25000


In [ ]:
#이미지를 숫자로 변환해야 함
# 1. opencv 라이브러리로 이미지 숫자화하기
# 2. tf.keras 이용하여 한큐에 처리하기(tf.keras.preprocessing.image_dataset_from_directory()) -> 이거로 하겠음
import tensorflow as tf
import shutil # 파일의 위치를 옮길 수 있도록 도와줌

# dataset이라는 폴더를 만들고 cat, dog이라는 폴더를 만들었다.

for i in os.listdir('/content/train/'): # [cat1.jpg, cat2.jpg, ... dog1.jpg, dog2.jpg...]
  if 'cat' in i: # 이름에 cat이 들어있으면, cat폴더로 이동
    shutil.copyfile('/content/train/' + i, '/content/dataset/cat/' + i)
  if 'dog' in i: # 이름에 dog이 들어있으면, dog폴더로 이동
    shutil.copyfile('/content/train/' + i, '/content/dataset/dog/' + i)

# tf.keras.preprocessing.image_dataset_from_directory()


In [ ]:
# %rm -rf dataset

In [ ]:
train_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/dataset/',
  image_size=(64,64), # 모든 데이터들을 64*64로 만든다. -> 전처리
  batch_size=32, # epoch(반복)를 돌릴 때, 이미지 2만장을 한번에 넣지 않고, batch 숫자만큼만 넣겠다는 뜻 (한번에 하지 않고 나눠서 넣겠다.)
  subset='training', # training의 이름을 가졌음 -> 데이터 셋의 80%를 가져감
  validation_split=0.2, # 20%의 데이터를 validation_data_set으로 쪼갠다.
  seed=1234
) # 폴더내의 이미지들을 바로 적용 가능한 dataset으로 만들어줌

# train_ds에는 -> ( (xxxxxxxxx[32개]), (yyyyyyyyy[32개])) 형태의 데이터가 저장된다.

val_ds = tf.keras.preprocessing.image_dataset_from_directory(
  '/content/dataset/',
  image_size=(64,64), 
  batch_size=32, 
  subset='validation', # validation의 이름을 가졌음 -> 데이터 셋의 20%를 가져감
  validation_split=0.2,
  seed=1234
) 

print(train_ds) # 32개씩 들어있는 batch들의 모음

def preprocessFunc(i, ans):
  i = tf.cast(i/255.0, tf.float32)
  return i, ans

train_ds = train_ds.map(preprocessFunc) # map함수 -> train_ds 데이터에 전부 () 안의 함수를 적용시킴
val_ds = val_ds.map(preprocessFunc)

# import matplotlib.pyplot as plt


# for i, ans in train_ds.take(1): # train_ds의 32개씩 들어있는 batch들에서 하나를 뽑아옴
#   print(i) # i -> shape=(32, 64, 64, 3)의 데이터 셋
#   print(ans) # ans -> shape=(32,) / 32개의 0 또는 1 로 구성된 데이터 셋
#   plt.imshow(i[0].numpy().astype('uint8')) # tensor를 numpy로 변환 / matplotlib는 numpy에서 작동하나봄
#   plt.show()

Found 25000 files belonging to 2 classes.
Using 20000 files for training.
Found 25000 files belonging to 2 classes.
Using 5000 files for validation.
<BatchDataset element_spec=(TensorSpec(shape=(None, 64, 64, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>


In [ ]:
model = tf.keras.Sequential([
    tf.keras.layers.experimental.preprocessing.RandomFlip('horizontal', input_shape=(64,64,3)),
    tf.keras.layers.experimental.preprocessing.RandomRotation(0.1),
    tf.keras.layers.experimental.preprocessing.RandomZoom(0.1),

    tf.keras.layers.Conv2D(32, (3,3), padding="same", activation='relu', input_shape=(64,64,3)),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Conv2D(64, (3,3), padding="same", activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Dropout(0.2), # overfitting 완화기능 -> 윗레이어 노드의 20%의 값을 0으로 만듬
    tf.keras.layers.Conv2D(128, (3,3), padding="same", activation='relu'),
    tf.keras.layers.MaxPooling2D((2,2)),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(128, activation="relu"),
    tf.keras.layers.Dropout(0.2),
    tf.keras.layers.Dense(1, activation="sigmoid") # 0~1사이 값이 출력 됨
])

model.summary()

model.compile(loss="binary_crossentropy", metrics=['accuracy'])
model.fit(train_ds, validation_data=(val_ds), epochs=5)

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 random_flip (RandomFlip)    (None, 64, 64, 3)         0         
                                                                 
 random_rotation (RandomRota  (None, 64, 64, 3)        0         
 tion)                                                           
                                                                 
 random_zoom (RandomZoom)    (None, 64, 64, 3)         0         
                                                                 
 conv2d (Conv2D)             (None, 64, 64, 32)        896       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 32, 32, 32)       0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 32, 32, 64)        1